# 학교 실습 1 : K-nearest neighbor 분류와 로지스틱 회귀 분류
- 문제 : 7개의 생선 종류를 분류
- KNN으로 확률 예측
- 로지스틱 회귀 이진 분류
- 로지스틱 회귀 다중 분류
- 파일 이름 : logisticregression.ipynb

## K-nearest neighbor
- classification 알고리즘의 일종
  - 새로운 데이터가 들어왔을 때 기존 데이터의 그룹(K개의 그룹) 중 어떤 그룹에 속하는지 분류하는 알고리즘

데이터 준비하기

In [2]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head() #처음 5개의 행을 출력

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


데이터 한번 보기

In [3]:
print(pd.unique(fish['Species'])) #물고기 종류의 고유값 출력

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [4]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy() 
#6개 중에서 Species를 타깃으로 하고 나머지 5개의 특성을 가지고 넘파이 배열을 만들고 fish_input에 저장

In [5]:
print(fish_input[:5]) #첫 5행을 출력

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [6]:
fish_target = fish['Species'].to_numpy() #Species가 타겟

훈련 세트와 테스트 세트로 구분하고 표준화 전처리

In [7]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)
 #훈련세트와 테스트 세트로 나누어 주는 함수(기본적으로 25%가 테스트세트)

In [8]:
from sklearn.preprocessing import StandardScaler #표준화 전처리

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

knn 분류기의 확률 예측

In [9]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3) #k=3
kn.fit(train_scaled, train_target) #훈련

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))   #점수 프린트

0.8907563025210085
0.85


In [10]:
print(kn.classes_)
 #KNeighborsClassifier에 의해 정렬된 타깃 값(자동으로 알파벳 순서로 매겨진다).

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [11]:
print(kn.predict(test_scaled[:5]))
 #테스트세트에 있는 5개 샘플의 타깃값을 예측

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


예측 확률의 값과 k=3일 때의 이웃 값

In [12]:
import numpy as np

proba = kn.predict_proba(test_scaled[:5]) #predict_proba : 예측확률 함수
print(np.round(proba, decimals=4))
#처음 5개의 예측 확률값, 첫번째의 경우 세번째가 확률이 100%로 Perch, 4번째의 경우 66,67%로 Perch

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


In [13]:
distances, indexes = kn.kneighbors(test_scaled[3:4])   
#4번째 Perch로 예측했던 것의 이웃분포를 프린트 해 보니 셋 중에 두개가 Perch, 그러므로 66.67% Perch가 33.33%라는 뜻
print(train_target[indexes])

[['Roach' 'Perch' 'Perch']]


분류와 회귀의 차이 : k-최근접 이웃 회귀
